In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from operator import itemgetter
import os

In [5]:
path = \
'/Users/shrutikaushal/Documents/GitHub/Prediction-of-commercial-insurance-payments-for-surgical-procedure-using-DataRobot/'

In [19]:
original_data = pd.read_csv(path+'Data_Files/JnJ_Files/priv_mcare_f_pay_2022Oct18.csv')
population_data = pd.read_csv(path+'Data_Files/Age_Sex_Data/ACSST1Y2021.S0101-Data.csv')

In [20]:
original_data.head(2)

,msa,year,site,group,priv_count,priv_pay_mean,priv_pay_median,priv_pay_iqr,mcare_count,mcare_los,mcare_pay_mean,mcare_pay_median,mcare_pay_sd,CBSA_NAME,State,lon,lat
0,10740.0,2018,Inpatient,breast reconstruction,0.0,NaN,NaN,NaN,NaN,4.0,22464.25667,16374.48,13087.222,"Albuquerque, NM",New Mexico,-106.650422,35.084386
1,10780.0,2018,Inpatient,breast reconstruction,0.0,17041.29,17041.29,0.0,NaN,2.0,8810.25000,8810.25,NaN,"Alexandria, LA",Louisiana,-92.445137,31.311294


In [21]:
population_data.head(2)

,GEO_ID,NAME,Estimate!!Total!!Total population,Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years),Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Sex ratio (males per 100 females)
0,310M600US10140,"Aberdeen, WA",76841,44.5,103.0
1,310M600US10180,"Abilene, TX",178608,34.4,106.1


In [22]:
population_data.drop(['GEO_ID'], axis = 1, inplace = True)
population_data.columns = ['CBSA_NAME','total_population','median_age','sex_ratio']

In [23]:
population_data.head()

,CBSA_NAME,total_population,median_age,sex_ratio
0,"Aberdeen, WA",76841,44.5,103.0
1,"Abilene, TX",178608,34.4,106.1
2,"Adrian, MI",98956,42.0,101.6
3,"Aguadilla-Isabela, PR",309515,45.3,92.8
4,"Akron, OH",700015,41.1,95.2


In [26]:
temp_df = pd.merge(original_data, population_data, on = 'CBSA_NAME', how = 'left')

In [32]:
print('Could not match MSA of {} number of observations from dataset'.format(\
                                            temp_df[temp_df.total_population.isnull()].shape[0]))

Could not match MSA of 6540 number of observations from dataset


In [77]:
no_match_msa = temp_df[temp_df.total_population.isnull()].CBSA_NAME.dropna().unique()

In [78]:
no_match_msa

array(['Prescott, AZ', 'Dayton, OH', 'San Rafael, CA',
       'Tacoma-Lakewood, WA', 'Dutchess County-Putnam County, NY',
       'Oakland-Hayward-Berkeley, CA', 'Anaheim-Santa Ana-Irvine, CA',
       'West Palm Beach-Boca Raton-Delray Beach, FL',
       'San Juan-BayamÃ³n-Caguas, PR',
       'San Francisco-Redwood City-South San Francisco, CA',
       'Miami-Miami Beach-Kendall, FL', 'Seattle-Bellevue-Everett, WA',
       'Los Angeles-Long Beach-Glendale, CA',
       'Silver Spring-Frederick-Rockville, MD',
       'Chicago-Naperville-Arlington Heights, IL',
       'Fort Lauderdale-Pompano Beach-Deerfield Beach, FL',
       'San GermÃ¡n, PR', 'MayagÃ¼ez, PR'], dtype=object)

**There are 654 observations with missing MSA values**

In [79]:
original_data[original_data.CBSA_NAME.isnull()]

,msa,year,site,group,priv_count,priv_pay_mean,priv_pay_median,priv_pay_iqr,mcare_count,mcare_los,mcare_pay_mean,mcare_pay_median,mcare_pay_sd,CBSA_NAME,State,lon,lat
22409,NaN,2019,Inpatient,clavicle fixation,1.0,61156.50,61156.50,0.0,NaN,3.0,6015.81,6015.81,NaN,NaN,NaN,NaN,NaN
26236,NaN,2018,Inpatient,lung ablation,1.0,38557.70,38557.70,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26243,NaN,2019,Inpatient,lung ablation,1.0,38241.00,38241.00,0.0,NaN,17.0,30480.91,30480.91,NaN,NaN,NaN,NaN,NaN
26277,NaN,2020,Outpatient,lung ablation,1.0,4924.05,4924.05,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26378,NaN,2020,Outpatient,laac,1.0,42592.10,42592.10,0.0,NaN,0.0,2398.87,2398.87,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132246,NaN,2018,ASC,post_TLS_fusion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132250,0.0,2019,ASC,post_TLS_fusion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132594,NaN,2019,ASC,post_TLS_fusion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132598,0.0,2020,ASC,post_TLS_fusion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
no_match_msa

array(['Prescott, AZ', 'Dayton, OH', 'San Rafael, CA',
       'Tacoma-Lakewood, WA', 'Dutchess County-Putnam County, NY',
       'Oakland-Hayward-Berkeley, CA', 'Anaheim-Santa Ana-Irvine, CA',
       'West Palm Beach-Boca Raton-Delray Beach, FL',
       'San Juan-BayamÃ³n-Caguas, PR',
       'San Francisco-Redwood City-South San Francisco, CA',
       'Miami-Miami Beach-Kendall, FL', 'Seattle-Bellevue-Everett, WA',
       'Los Angeles-Long Beach-Glendale, CA',
       'Silver Spring-Frederick-Rockville, MD',
       'Chicago-Naperville-Arlington Heights, IL',
       'Fort Lauderdale-Pompano Beach-Deerfield Beach, FL',
       'San GermÃ¡n, PR', 'MayagÃ¼ez, PR'], dtype=object)

In [85]:
#trying to match MSAs to population dataset
match_dict = {}
for x in no_match_msa:
    y = x.split(',')[0]
    if '-' in y:
        y = y.split('-')
    else:
        y = y.split(' ')
    match_dict[x] = [z for z in population_data.CBSA_NAME.dropna().unique()\
                     if set(z.split()).intersection(set(y))!=set()]

In [102]:
{key:match_dict[key] for key in match_dict.keys() if match_dict[key]!=[]}

{'Prescott, AZ': ['Prescott Valley-Prescott, AZ'],
 'San Rafael, CA': ['San Angelo, TX',
  'San Antonio-New Braunfels, TX',
  'San Diego-Chula Vista-Carlsbad, CA',
  'San Francisco-Oakland-Berkeley, CA',
  'San Germán, PR',
  'San Jose-Sunnyvale-Santa Clara, CA',
  'San Juan-Bayamón-Caguas, PR',
  'San Luis Obispo-Paso Robles, CA'],
 'San GermÃ¡n, PR': ['San Angelo, TX',
  'San Antonio-New Braunfels, TX',
  'San Diego-Chula Vista-Carlsbad, CA',
  'San Francisco-Oakland-Berkeley, CA',
  'San Germán, PR',
  'San Jose-Sunnyvale-Santa Clara, CA',
  'San Juan-Bayamón-Caguas, PR',
  'San Luis Obispo-Paso Robles, CA']}

In [103]:
# saving data
temp_df.to_csv(path+'Data_Files/Added_Data/original_w_population.csv', index = False)